# 1. Diffusion Model Fundamentals

**From Noise to Images: Understanding the Core of Modern AI Art**

---

Diffusion models are the backbone of the most powerful image generation systems today -- **Stable Diffusion**, **DALL-E 3**, **Midjourney**, and **FLUX**. Rather than generating images in a single forward pass (like GANs), diffusion models work by *learning to reverse a gradual noising process*.

This notebook covers the mathematical and intuitive fundamentals:

1. The forward and reverse diffusion processes
2. Noise schedules and why they matter
3. The training objective (epsilon prediction)
4. Sampling algorithms -- from 1000-step DDPM to 4-step Flow Matching

**Requirements**: Only `numpy` and `matplotlib` -- no GPU needed.

---

## What Is a Diffusion Model?

A diffusion model defines two complementary processes:

### Forward Process $q$ (Noising)

Gradually add **Gaussian noise** to real data $x_0$ over $T$ timesteps until the result is indistinguishable from pure random noise. Each step is a small perturbation:

$$q(x_t \mid x_{t-1}) = \mathcal{N}\!\left(x_t;\ \sqrt{1 - \beta_t}\, x_{t-1},\ \beta_t \mathbf{I}\right)$$

where $\beta_t$ is a small variance that increases over time. Thanks to the reparameterization trick, we can jump directly to any timestep $t$:

$$q(x_t \mid x_0) = \mathcal{N}\!\left(x_t;\ \sqrt{\bar{\alpha}_t}\, x_0,\ (1 - \bar{\alpha}_t)\mathbf{I}\right) \qquad \text{where } \bar{\alpha}_t = \prod_{s=1}^{t}(1 - \beta_s)$$

### Reverse Process $p_\theta$ (Denoising)

A neural network learns to **reverse** each noising step, recovering the original data from pure noise:

$$p_\theta(x_{t-1} \mid x_t) = \mathcal{N}\!\left(x_{t-1};\ \mu_\theta(x_t, t),\ \Sigma_\theta(x_t, t)\right)$$

### Intuition

> **Analogy**: Imagine a photograph that has been progressively damaged -- first a little static, then more, until it is pure snow. The forward process *is* that progressive damage. The reverse process is a neural network that has studied millions of photos and learned, step by step, how to *restore* the image at every level of damage.

| Forward Process | Reverse Process |
|---|---|
| Fixed (no learnable parameters) | Learned by the neural network |
| Adds noise according to schedule $\beta_t$ | Predicts and removes noise |
| $x_0 \rightarrow x_1 \rightarrow \cdots \rightarrow x_T \sim \mathcal{N}(0, \mathbf{I})$ | $x_T \rightarrow x_{T-1} \rightarrow \cdots \rightarrow x_0$ |

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ── Create a synthetic "image" (colorful gradient pattern) ──────────────────
def create_sample_image(size=128):
    """Create a colorful gradient pattern as our sample image."""
    x = np.linspace(0, 1, size)
    y = np.linspace(0, 1, size)
    xx, yy = np.meshgrid(x, y)
    r = np.sin(2 * np.pi * xx) * 0.5 + 0.5
    g = np.cos(2 * np.pi * yy) * 0.5 + 0.5
    b = np.sin(2 * np.pi * (xx + yy)) * 0.5 + 0.5
    return np.stack([r, g, b], axis=-1)

image = create_sample_image()

# ── Define a linear noise schedule ─────────────────────────────────────────
T = 1000
betas = np.linspace(1e-4, 0.02, T)
alphas = 1 - betas
alpha_cumprod = np.cumprod(alphas)

# ── Visualize progressive noising at selected timesteps ───────────────────
timesteps = [0, 100, 250, 500, 750, 999]

fig, axes = plt.subplots(1, len(timesteps), figsize=(18, 3.2))
np.random.seed(0)

for i, t in enumerate(timesteps):
    sqrt_alpha = np.sqrt(alpha_cumprod[t])
    sqrt_one_minus_alpha = np.sqrt(1 - alpha_cumprod[t])
    noise = np.random.randn(*image.shape)
    noisy = sqrt_alpha * image + sqrt_one_minus_alpha * noise
    noisy = np.clip(noisy, 0, 1)

    axes[i].imshow(noisy)
    signal_pct = alpha_cumprod[t] * 100
    axes[i].set_title(
        f't = {t}\n$\\bar{{\\alpha}}_t$ = {alpha_cumprod[t]:.3f} ({signal_pct:.0f}% signal)',
        fontsize=10
    )
    axes[i].axis('off')

fig.suptitle(
    'Forward Diffusion Process: Gradually Adding Noise',
    fontsize=14, fontweight='bold', y=1.02
)
plt.tight_layout()
plt.show()

---

## Noise Schedules

The **noise schedule** $\{\beta_t\}_{t=1}^{T}$ controls how quickly information is destroyed during the forward process. It is one of the most important design choices in a diffusion model.

### Linear Schedule (DDPM, Ho et al. 2020)

$\beta_t$ increases linearly from $\beta_1 = 10^{-4}$ to $\beta_T = 0.02$. Simple and effective, but destroys too much information in the early timesteps, which can hurt generation quality.

### Cosine Schedule (Nichol & Dhariwal, 2021)

Designed so that $\bar{\alpha}_t$ follows a cosine curve, providing **smoother degradation**. The signal is preserved longer in the early steps and decays more gracefully:

$$\bar{\alpha}_t = \frac{f(t)}{f(0)}, \qquad f(t) = \cos\!\left(\frac{t/T + s}{1 + s} \cdot \frac{\pi}{2}\right)^{\!2}$$

where $s = 0.008$ is a small offset to prevent $\beta_t$ from being too small near $t = 0$.

### Why It Matters

- A schedule that destroys information **too quickly** wastes timestep budget on near-pure noise.
- A schedule that is **too slow** requires more steps and compute.
- The cosine schedule achieves better FID scores because it spends more timesteps in the perceptually meaningful mid-noise range.

In [ ]:
T = 1000
t = np.arange(T)

# ── Linear schedule ────────────────────────────────────────────────────────
betas_linear = np.linspace(1e-4, 0.02, T)
alphas_linear = 1 - betas_linear
alpha_cumprod_linear = np.cumprod(alphas_linear)

# ── Cosine schedule ────────────────────────────────────────────────────────
def cosine_schedule(T, s=0.008):
    """Cosine noise schedule (Nichol & Dhariwal, 2021)."""
    steps = np.arange(T + 1)
    f = np.cos((steps / T + s) / (1 + s) * np.pi / 2) ** 2
    alphas_cumprod = f / f[0]
    betas = 1 - alphas_cumprod[1:] / alphas_cumprod[:-1]
    return np.clip(betas, 0, 0.999), alphas_cumprod[1:]

betas_cosine, alpha_cumprod_cosine = cosine_schedule(T)

# ── Plot comparison ────────────────────────────────────────────────────────
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Left: beta_t
ax1.plot(t, betas_linear, label='Linear', linewidth=2, color='#2196F3')
ax1.plot(t, betas_cosine, label='Cosine', linewidth=2, color='#FF5722')
ax1.set_xlabel('Timestep $t$', fontsize=12)
ax1.set_ylabel('$\\beta_t$', fontsize=12)
ax1.set_title('Noise Schedule: $\\beta_t$ over Time', fontsize=13, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# Right: cumulative alpha
ax2.plot(t, alpha_cumprod_linear, label='Linear', linewidth=2, color='#2196F3')
ax2.plot(t, alpha_cumprod_cosine, label='Cosine', linewidth=2, color='#FF5722')
ax2.set_xlabel('Timestep $t$', fontsize=12)
ax2.set_ylabel('$\\bar{\\alpha}_t$ (cumulative product)', fontsize=12)
ax2.set_title('Signal Retention: $\\bar{\\alpha}_t$ over Time', fontsize=13, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

# Annotate the key difference
ax2.annotate(
    'Cosine preserves\nmore signal early on',
    xy=(200, alpha_cumprod_cosine[200]), xytext=(350, 0.85),
    arrowprops=dict(arrowstyle='->', color='#FF5722', lw=1.5),
    fontsize=10, color='#FF5722', fontweight='bold'
)

plt.suptitle(
    'Linear vs Cosine Noise Schedules',
    fontsize=14, fontweight='bold', y=1.02
)
plt.tight_layout()
plt.show()

---

## The Training Objective

### Epsilon Prediction (DDPM Loss)

Instead of predicting the clean image $x_0$ directly, the network predicts the **noise** $\epsilon$ that was added. The loss is a simple mean squared error:

$$\mathcal{L} = \mathbb{E}_{x_0,\, t,\, \epsilon}\!\left[\left\| \epsilon - \epsilon_\theta(x_t, t) \right\|^2\right]$$

### Training Loop (simplified)

```
repeat:
    1. Sample x_0 from the training data
    2. Sample t ~ Uniform({1, ..., T})
    3. Sample noise: epsilon ~ N(0, I)
    4. Compute noisy image: x_t = sqrt(alpha_bar_t) * x_0 + sqrt(1 - alpha_bar_t) * epsilon
    5. Predict noise: epsilon_hat = model(x_t, t)
    6. Compute loss: L = MSE(epsilon, epsilon_hat)
    7. Backpropagate and update model weights
```

### Why Predict Noise Instead of the Image?

- **Numerically stable**: Noise is zero-mean and unit-variance regardless of timestep.
- **Equivalent formulations**: Predicting $\epsilon$ is mathematically equivalent to predicting $x_0$ or the score $\nabla_{x_t} \log q(x_t)$ -- but epsilon prediction trains most reliably in practice.
- **Connection to score matching**: $\epsilon_\theta \approx -\sqrt{1 - \bar{\alpha}_t}\, \nabla_{x_t} \log q(x_t \mid x_0)$, linking diffusion models to score-based generative models.

In [ ]:
# ── Training Loop Diagram ──────────────────────────────────────────────────
# Visualize the DDPM training pipeline as a flowchart using matplotlib

fig, ax = plt.subplots(figsize=(14, 7))
ax.set_xlim(0, 14)
ax.set_ylim(0, 7)
ax.axis('off')

# ── Style constants ────────────────────────────────────────────────────────
box_color = '#E3F2FD'
box_edge = '#1565C0'
highlight_color = '#FFF3E0'
highlight_edge = '#E65100'
arrow_color = '#37474F'
text_color = '#212121'

def draw_box(ax, x, y, w, h, text, color=box_color, edge=box_edge, fontsize=10):
    """Draw a rounded rectangle with centered text."""
    from matplotlib.patches import FancyBboxPatch
    box = FancyBboxPatch(
        (x - w/2, y - h/2), w, h,
        boxstyle='round,pad=0.15', facecolor=color,
        edgecolor=edge, linewidth=2
    )
    ax.add_patch(box)
    ax.text(x, y, text, ha='center', va='center', fontsize=fontsize,
            color=text_color, fontweight='bold', wrap=True)

def draw_arrow(ax, x1, y1, x2, y2):
    """Draw an arrow between two points."""
    ax.annotate(
        '', xy=(x2, y2), xytext=(x1, y1),
        arrowprops=dict(
            arrowstyle='->', color=arrow_color,
            lw=2, connectionstyle='arc3,rad=0'
        )
    )

# ── Title ───────────────────────────────────────────────────────────────────
ax.text(7, 6.6, 'DDPM Training Loop', ha='center', fontsize=16,
        fontweight='bold', color='#0D47A1')

# ── Row 1: Inputs ──────────────────────────────────────────────────────────
draw_box(ax, 2.5, 5.5, 3.2, 0.7, '1. Sample $x_0$ from data')
draw_box(ax, 7.0, 5.5, 3.2, 0.7, '2. Sample $t \\sim U\\{1..T\\}$')
draw_box(ax, 11.5, 5.5, 3.2, 0.7, '3. Sample $\\epsilon \\sim \\mathcal{N}(0, I)$')

# ── Arrows down ────────────────────────────────────────────────────────────
draw_arrow(ax, 2.5, 5.1, 7.0, 4.35)
draw_arrow(ax, 7.0, 5.1, 7.0, 4.35)
draw_arrow(ax, 11.5, 5.1, 7.0, 4.35)

# ── Row 2: Compute noisy image ─────────────────────────────────────────────
draw_box(ax, 7.0, 4.0, 8.5, 0.7,
         '4. Compute $x_t = \\sqrt{\\bar{\\alpha}_t}\\, x_0 '
         '+ \\sqrt{1 - \\bar{\\alpha}_t}\\, \\epsilon$',
         fontsize=11)

draw_arrow(ax, 7.0, 3.6, 7.0, 2.85)

# ── Row 3: Network prediction ──────────────────────────────────────────────
draw_box(ax, 7.0, 2.5, 6.0, 0.7,
         '5. Predict $\\hat{\\epsilon} = \\epsilon_\\theta(x_t,\\, t)$',
         color=highlight_color, edge=highlight_edge, fontsize=11)

draw_arrow(ax, 7.0, 2.1, 7.0, 1.35)

# ── Row 4: Loss & update ───────────────────────────────────────────────────
draw_box(ax, 7.0, 1.0, 8.0, 0.7,
         '6. Loss $\\mathcal{L} = \\|\\epsilon - \\hat{\\epsilon}\\|^2$'
         '        7. Backprop & update $\\theta$',
         color='#E8F5E9', edge='#2E7D32', fontsize=11)

# ── Loop-back arrow ────────────────────────────────────────────────────────
ax.annotate(
    '', xy=(0.5, 5.5), xytext=(0.5, 1.0),
    arrowprops=dict(
        arrowstyle='->', color='#B71C1C', lw=2.5,
        connectionstyle='arc3,rad=0'
    )
)
ax.text(0.3, 3.25, 'repeat', rotation=90, ha='center', va='center',
        fontsize=11, fontweight='bold', color='#B71C1C')

plt.tight_layout()
plt.show()

---

## Sampling Algorithms

Once the model is trained, we generate images by running the **reverse process** -- starting from pure noise $x_T \sim \mathcal{N}(0, \mathbf{I})$ and iteratively denoising.

The choice of **sampler** controls the trade-off between quality and speed:

| Sampler | Steps | Type | Key Idea |
|---------|-------|------|----------|
| **DDPM** (Ho et al., 2020) | 1000 | Stochastic | Original formulation; faithful but slow |
| **DDIM** (Song et al., 2020) | 50--100 | Deterministic | Reinterprets diffusion as an ODE; skip steps |
| **Euler** | 20--50 | ODE solver | Simple first-order discretization |
| **DPM-Solver** (Lu et al., 2022) | 20--25 | Higher-order ODE | Exponential integrator; fast convergence |
| **Flow Matching** (Lipman et al., 2023) | 4--28 | ODE / straight paths | Used by SD3 and FLUX; linear trajectories |

### Key Insight

Modern architectures like **Stable Diffusion 3** and **FLUX** use **Flow Matching** (also called Rectified Flows), which learns *straight-line* trajectories from noise to data. Because the paths are straighter, fewer discretization steps are needed -- achieving high quality in as few as **4 steps**.

This is a paradigm shift: early diffusion models needed 1000 steps; state-of-the-art models now need fewer than 30.

In [ ]:
# ── Sampling Strategies: 1-D denoising trajectories ────────────────────────

np.random.seed(42)

def simulate_denoising(n_steps, noise_start=3.0, target=1.0):
    """Simulate a 1-D denoising trajectory from noise to a target value."""
    trajectory = [noise_start]
    for i in range(n_steps):
        progress = (i + 1) / n_steps
        current = trajectory[-1]
        remaining = n_steps - i
        step = (target - current) / remaining
        noise = np.random.randn() * (1 - progress) * 0.5
        trajectory.append(current + step + noise)
    return np.array(trajectory)

samplers = {
    'DDPM (1000 steps)': 1000,
    'DDIM (50 steps)': 50,
    'Euler (20 steps)': 20,
    'FLUX Flow Matching (4 steps)': 4,
}

colors = ['#1565C0', '#2E7D32', '#E65100', '#6A1B9A']

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for ax, (name, steps), color in zip(axes, samplers.items(), colors):
    trajectory = simulate_denoising(steps)
    x_axis = np.linspace(0, 1, len(trajectory))  # normalize to [0, 1]

    marker = 'o' if steps <= 50 else ''
    ms = max(2, 10 - steps // 10)

    ax.plot(x_axis, trajectory, color=color, marker=marker,
            markersize=ms, alpha=0.8, linewidth=2)
    ax.axhline(y=1.0, color='#D32F2F', linestyle='--', alpha=0.6,
               label='Target signal', linewidth=1.5)
    ax.fill_between(x_axis, trajectory, 1.0, alpha=0.07, color=color)

    ax.set_title(name, fontsize=13, fontweight='bold', color=color)
    ax.set_xlabel('Progress (0 = pure noise, 1 = final)', fontsize=10)
    ax.set_ylabel('Value', fontsize=10)
    ax.legend(fontsize=9, loc='upper right')
    ax.grid(True, alpha=0.3)
    ax.set_ylim(-0.5, 4.0)

plt.suptitle(
    'Reverse Process: Different Sampling Strategies',
    fontsize=14, fontweight='bold', y=1.01
)
plt.tight_layout()
plt.show()

---

## Key Takeaways

<div style="background-color: #E3F2FD; padding: 20px; border-radius: 10px; border-left: 5px solid #1565C0;">

1. **Diffusion models learn to reverse a noising process** -- the forward process destroys information by adding Gaussian noise; the reverse process (a neural network) learns to reconstruct it.

2. **The noise schedule controls the speed of information destruction.** Cosine schedules outperform linear schedules by preserving perceptually meaningful signal longer.

3. **The training objective is simple**: predict the noise $\epsilon$ that was added, then minimize MSE. This is equivalent to score matching.

4. **Modern samplers dramatically reduce the number of steps needed.** From 1000 steps (DDPM) to 4 steps (Flow Matching in FLUX) -- a 250x speedup.

5. **Next notebook**: How the neural network architecture evolved -- from the original **UNet** to **DiT** (Diffusion Transformer) to **MMDiT** (used in SD3 and FLUX).

</div>

---

## References

1. **Ho, J., Jain, A., & Abbeel, P.** (2020). *Denoising Diffusion Probabilistic Models*. NeurIPS 2020. [arXiv:2006.11239](https://arxiv.org/abs/2006.11239)

2. **Song, J., Meng, C., & Ermon, S.** (2020). *Denoising Diffusion Implicit Models*. ICLR 2021. [arXiv:2010.02502](https://arxiv.org/abs/2010.02502)

3. **Nichol, A. Q. & Dhariwal, P.** (2021). *Improved Denoising Diffusion Probabilistic Models*. ICML 2021. [arXiv:2102.09672](https://arxiv.org/abs/2102.09672)

4. **Lu, C., Zhou, Y., Bao, F., Chen, J., Li, C., & Zhu, J.** (2022). *DPM-Solver: A Fast ODE Solver for Diffusion Probabilistic Model Sampling*. NeurIPS 2022. [arXiv:2206.00927](https://arxiv.org/abs/2206.00927)

5. **Lipman, Y., Chen, R. T. Q., Ben-Hamu, H., Nickel, M., & Le, M.** (2023). *Flow Matching for Generative Modeling*. ICLR 2023. [arXiv:2210.02747](https://arxiv.org/abs/2210.02747)